# Import library

In [1]:
%load_ext autoreload
%autoreload 2

# speed up the loading of the training data
import cv2
import os
import numpy as np
import torch as th
import itertools
import torch.nn as nn
import torch.nn.functional as F
import torch.nn.init as init
from onlineTripletloss import *
from selector import *
from model_SAGAN1_1 import NetG, NetD, NetA
# from model_SAGAN2_Triplet import NetG, NetD, NetA
# from model_WGANGP import NetG, NetD, NetA
# from model_WGAN import NetG, NetD, NetA
# from model_siGAN import NetG, NetD, NetA
# from dataset2Loader import CASIABDataset
# from dataset2Loader_newtriplet import CASIABDataset
from dataset2Loader_triplet import CASIABDataset
import torch.optim as optim
import visdom
from torchvision.utils import make_grid
# Data_Dir = '../GaitRecognition/DatasetB_GEI_64x64_allseq/'
Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
Model_Name = 'Model_64x64_TripletSAGAN_90_trial11'
Model_dir = './Transform_Model/'+ Model_Name
if not os.path.isdir(Model_dir):
    os.mkdir(Model_dir)

In [ ]:

Data_Dir = '../GaitRecognition/GEI_CASIA_B/gei/'
import os

for r, d, files in os.walk(Data_Dir):
    print(r)
    print(len(d))
    print(len(files))
    
# cpt = sum([len(files) for r, d, files in os.walk(Data_Dir)])
# cpt = sum([len(d) for r, d, files in os.walk(Data_Dir)])
# print(cpt)
# list = os.listdir(Data_Dir) # dir is your directory path
# number_files = len(list)
# print(number_files)

# import fnmatch
# print(len(fnmatch.filter(os.listdir(Data_Dir), '*.png')))

In [ ]:
dataset = CASIABDataset(data_dir=Data_Dir)
ass_label, noass_label, img = dataset.getbatch(16)

## iteration 刪

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
        init.xavier_normal_(tensor, gain=1.)
#         init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir)

iteration = 0
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 128

label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print('Training starts')
while iteration < 30000:
    ass_label, noass_label, img = dataset.getbatch(batchSize)
    ass_label = ass_label.to(device).to(th.float32)
    noass_label = noass_label.to(device).to(th.float32)
    img = img.to(device).to(th.float32)
    
    # update D
    lossD = 0
    optimD.zero_grad()
    output = netd(ass_label)
    label.fill_(real_label)
    lossD_real1 = F.binary_cross_entropy(output, label)
    lossD += lossD_real1.item()
    lossD_real1.backward()

    label.fill_(real_label)
    output1 = netd(noass_label)
    lossD_real2 = F.binary_cross_entropy(output1, label)
    lossD += lossD_real2.item()
    lossD_real2.backward()

    fake = netg(img).detach()
    label.fill_(fake_label)
    output2 = netd(fake)
    lossD_fake = F.binary_cross_entropy(output2, label)
    lossD += lossD_fake.item()
    lossD_fake.backward()

    optimD.step()
    
    # update A
    lossA = 0
    optimA.zero_grad()
    assd = th.cat((img, ass_label), 1)
    noassd = th.cat((img, noass_label), 1)
    fake = netg(img).detach()
    faked = th.cat((img, fake), 1)

    label.fill_(real_label)
    output1 = neta(assd)
    lossA_real1 = F.binary_cross_entropy(output1, label)
    lossA += lossA_real1.item()
    lossA_real1.backward()

    label.fill_(fake_label)
    output = neta(noassd)
    lossA_real2 = F.binary_cross_entropy(output, label)
    lossA += lossA_real2.item()
    lossA_real2.backward()

    label.fill_(fake_label)
    output = neta(faked)
    lossA_fake = F.binary_cross_entropy(output, label)
    lossA += lossA_fake.item()
    lossA_fake.backward()
    
    optimA.step()
    
    # update G
    lossG = 0
    optimG.zero_grad()
    fake = netg(img)
    output = netd(fake)

    label.fill_(real_label)
    lossGD = F.binary_cross_entropy(output, label)
    lossG += lossGD.item()
    lossGD.backward(retain_graph=True)

    faked = th.cat((img, fake), 1)
    output = neta(faked)
    label.fill_(real_label)
    lossGA = F.binary_cross_entropy(output, label)
    lossG += lossGA.item()
    lossGA.backward()
    
    optimG.step()

    iteration += 1
    
    if iteration % 20 == 0:
        with th.no_grad():
            netg.eval()  #切換
            fake = netg(img) 25
            netg.train() #切換回去
        fake = (fake + 1) / 2 * 255
        real = (ass_label + 1) / 2 * 255
        ori = (img + 1) / 2 * 255
        al = th.cat((fake, real, ori), 2)
        display = make_grid(al, 20).cpu().numpy()
        if win1 is None:
            win1 = vis.image(display,
                             opts=dict(title="train", caption='train'))
        else:
            vis.image(display, win=win1)
    if iteration % 1000 == 0 or iteration==500:
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_Name+'/snapshot'+ Model_Name +'_%d.t7' % iteration)

    if iteration % 100==0 or iteration==10 :
        if win is None:
            win = vis.line(X=np.array([[iteration, iteration,
                                        iteration]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='iterations',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[iteration, iteration,
                                  iteration]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')
            
        with open(Model_Name+"/snapshot_log.txt", "a") as myfile:
            myfile.write('iter = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            iteration, lossG/2, lossA/3, lossD/3
        ))
        print('iter = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            iteration, lossG/2, lossA/3, lossD/3
        ))


## Epoch

In [ ]:
dataset = CASIABDataset(data_dir=Data_Dir)
train_loader = th.utils.data.DataLoader(dataset, batch_size=128, shuffle=True, num_workers=2, pin_memory=False)
train_loader.next()

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 0
n_g = 0
n_d = 0


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=glr/2)
optimD = optim.Adam(netd.parameters(), lr=dlr/3)
optimA = optim.Adam(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

print('Training starts')
low_loss = 10
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#     for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
#         ass_img = ass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
#         noass_img = noass_img.to(device).to(th.float32)

        if i % n_g ==0:
            # update D
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake = netg(img).detach()
            label.fill_(fake_label)
            output2 = netd(fake)
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            fake = netg(img).detach()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()

        if i % n_d ==0: 
            # update G
            lossG = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

            label.fill_(real_label)
            lossGD = F.binary_cross_entropy(output, label)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
            label.fill_(real_label)
            lossGA = F.binary_cross_entropy(output, label)
            lossG += lossGA.item()
            lossGA.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:  
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
    
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        



## update k times

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 0
n_g = 2
n_d = 0


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=glr/2)
optimD = optim.Adam(netd.parameters(), lr=dlr/3)
optimA = optim.Adam(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10
print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#     for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
#         noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
#         ass_img = ass_img.to(device).to(th.float32)
        
        if i % n_g ==0:
            # update D
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake = netg(img).detach()
            label.fill_(fake_label)
            output2 = netd(fake)
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
        if i % n_d ==0: 
            # update G
            lossG = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

            label.fill_(real_label)
            lossGD = F.binary_cross_entropy(output, label)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
            label.fill_(real_label)
            lossGA = F.binary_cross_entropy(output, label)
            lossG += lossGA.item()
            lossGA.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))    
        

## model changed +Dp

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netp = NetP(nc=1)
neta = NetA(nc=1)
netd = NetD(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netp.children())[0].children(),
    list(neta.children())[0].children(),
    list(netd.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'

netg = netg.to(device)
netp = netp.to(device)
neta = neta.to(device)
netd = netd.to(device)
netg.train()
netp.train()
neta.train()
netd.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
optimP = optim.Adam(netp.parameters(), lr=lr/3)
optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, target = {} \n'.format(
            epoches, lr, batchSize, target))

print('Training starts')
low_loss = 10
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, ass_img, img, noass_img) in enumerate(train_loader): 
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        
        # update D
        lossD = 0
        optimD.zero_grad()
        output = netd(ass_label)
        label.fill_(real_label)
        lossD_real1 = F.binary_cross_entropy(output, label)
        lossD += lossD_real1.item()
        lossD_real1.backward()

        label.fill_(real_label)
        output1 = netd(noass_label)
        lossD_real2 = F.binary_cross_entropy(output1, label)
        lossD += lossD_real2.item()
        lossD_real2.backward()

        fake = netg(img).detach()
        label.fill_(fake_label)
        output2 = netd(fake)
        lossD_fake = F.binary_cross_entropy(output2, label)
        lossD += lossD_fake.item()
        lossD_fake.backward()

        optimD.step()


        # update P
        lossP = 0
        optimP.zero_grad()
        fake = netg(img).detach()
        assd = th.cat((img, ass_img), 1)
        noassd = th.cat((img, noass_img), 1)
        faked = th.cat((img, fake), 1)

        label.fill_(real_label)
        output1 = neta(assd)
        lossP_real1 = F.binary_cross_entropy(output1, label)
        lossP += lossP_real1.item()/2
        lossP_real1 = lossP_real1/2
        lossP_real1.backward()

        label.fill_(fake_label)
        output = neta(noassd)
        lossP_real2 = F.binary_cross_entropy(output, label)
        lossP += lossP_real2.item()/2
        lossP_real2 = lossP_real2/2
        lossP_real2.backward()

        label.fill_(fake_label)
        output = neta(faked)
        lossP_fake = F.binary_cross_entropy(output, label)
        lossP += lossP_fake.item()/2
        lossP_fake = lossP_fake/2
        lossP_fake.backward()

        optimP.step()
    

        # update A
        lossA = 0
        optimA.zero_grad()
        fake = netg(img).detach()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        faked = th.cat((img, fake), 1)

        label.fill_(real_label)
        output1 = neta(assd)
        lossA_real1 = F.binary_cross_entropy(output1, label)
        lossA += lossA_real1.item()
        lossA_real1.backward()

        label.fill_(fake_label)
        output = neta(noassd)
        lossA_real2 = F.binary_cross_entropy(output, label)
        lossA += lossA_real2.item()
        lossA_real2.backward()

        label.fill_(fake_label)
        output = neta(faked)
        lossA_fake = F.binary_cross_entropy(output, label)
        lossA += lossA_fake.item()
        lossA_fake.backward()

        optimA.step()

        # update G
        lossG = 0
        optimG.zero_grad()
        
        fake = netg(img)
        output = netd(fake)
        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward(retain_graph=True)
        
        faked = th.cat((img, fake), 1)
        output = netp(faked)
        label.fill_(real_label)
        lossGP = F.binary_cross_entropy(output, label)
        lossG += lossGP.item()/2
        lossGP = lossGP/2
        lossGP.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch, epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3, lossP/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD', 'lossP']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch, epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3, lossP/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
            
    if (epoch>= 300) and epoch%20==0:  
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict(),
            'netP': netp.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
    
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict(),
            'netP': netp.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, ErrP = {}  \n'.format(
            epoch, lossG/2, lossA/3, lossD/3, lossP/3
        ))
        



## model only D_A

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
epoches = 700
lr = 0.00002
# lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 32
g_k = 2
target = '090'

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
# netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr/2)
# optimD = optim.Adam(netd.parameters(), lr=lr/3)
optimA = optim.Adam(neta.parameters(), lr=lr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, lr = {}, batchsize = {}, g_k = {}, target={} \n'.format(
            epoches, lr, batchSize, g_k, target))

low_loss = 10
print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#     for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
#         noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
#         ass_img = ass_img.to(device).to(th.float32)
        
        if i % g_k ==0:
#             # update D
#             lossD = 0
#             optimD.zero_grad()
#             output = netd(ass_label)
#             label.fill_(real_label)
#             lossD_real1 = F.binary_cross_entropy(output, label)
#             lossD += lossD_real1.item()
#             lossD_real1.backward()

#             label.fill_(real_label)
#             output1 = netd(noass_label)
#             lossD_real2 = F.binary_cross_entropy(output1, label)
#             lossD += lossD_real2.item()
#             lossD_real2.backward()

#             fake = netg(img).detach()
#             label.fill_(fake_label)
#             output2 = netd(fake)
#             lossD_fake = F.binary_cross_entropy(output2, label)
#             lossD += lossD_fake.item()
#             lossD_fake.backward()

#             optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
#         if i % k ==0: 
        # update G
        lossG = 0
        optimG.zero_grad()
        fake = netg(img)
        output = netd(fake)

        label.fill_(real_label)
        lossGD = F.binary_cross_entropy(output, label)
        lossG += lossGD.item()
        lossGD.backward(retain_graph=True)

        faked = th.cat((img, fake), 1)
        output = neta(faked)
        label.fill_(real_label)
        lossGA = F.binary_cross_entropy(output, label)
        lossG += lossGA.item()
        lossGA.backward()

        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        ]]),
                           Y=np.array([[lossG/2, lossA/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  ]]),
                     Y=np.array([[lossG/2, lossA/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {} \n'.format(
            epoch, lossG/2, lossA/3,
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}'.format(
            epoch, lossG/2, lossA/3, 
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}\n'.format(
            epoch, lossG/2, lossA/3
        ))    
        

# GaitGAN and triplet 

In [ ]:
%load_ext autoreload
%autoreload 2

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '036'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 5
n_g = 0
n_d = 0


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=glr/2)
optimD = optim.Adam(netd.parameters(), lr=dlr/3)
optimA = optim.Adam(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        if i % n_g==0:
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake, _ = netg(img)
            label.fill_(fake_label)
            output2 = netd(fake.detach())
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake,_ = netg(img)
            faked = th.cat((img, fake), 1)

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked.detach())
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
        if i % n_d ==0: 
            # update G
            lossG = 0
            optimG.zero_grad()
            fake, A = netg(img)
            output = netd(fake)

            label.fill_(real_label)
            lossGD = F.binary_cross_entropy(output, label)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
            label.fill_(real_label)
            lossGA = F.binary_cross_entropy(output, label)
            lossG += lossGA.item()
            lossGA.backward()

            # constrain on generator
            fake_ass, P = netg(ass_label)
            fake_noass, N = netg(noass_label)
            lossTriplet = F.triplet_margin_loss(fake, fake_ass, fake_noass, margin = margin)
            lossG += lossTriplet.item()
            lossTriplet.backward()

            # constrain on encoder
    #         __, P = netg(ass_label)
    #         __, N = netg(noass_label)
    #         lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin)
    #         lossG += lossTriplet.item()
    #         lossTriplet.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
    


# GaitGAN k times and triplet   

In [ ]:
%load_ext autoreload
%autoreload 2

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 10
n_g = 2
n_d = 2


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=glr/3)
optimD = optim.Adam(netd.parameters(), lr=dlr/3)
optimA = optim.Adam(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         com_img = th.cat((noass_img, img, ass_img), 0)
#         com_label = th.cat(( label_neg, label_anc, label_pos), 0)
#         com_img = com_img.to(device).to(th.float32)
#         com_label = com_label.to(device).to(th.float32)
        
#         if(i ==0):
#             print(label_neg,label_anc,label_pos)
#             print(com_label)
#         print("shape",ass_label.shape,noass_label.shape,img.shape,com_img.shape, com_label.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)
        
        # update D
        if i % n_g==0:
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
            label.fill_(real_label)
            lossD_real1 = F.binary_cross_entropy(output, label)
            lossD += lossD_real1.item()
            lossD_real1.backward()

            label.fill_(real_label)
            output1 = netd(noass_label)
            lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake, _ = netg(img)
            label.fill_(fake_label)
            output2 = netd(fake.detach()) #需要 detach 因為不希望更新fake的參數
            lossD_fake = F.binary_cross_entropy(output2, label)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake,_ = netg(img)
            faked = th.cat((img, fake.detach()), 1)  #需要 detach 因為不希望更新fake的參數

            label.fill_(real_label)
            output1 = neta(assd)
            lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA += lossA_real1.item()
            lossA_real1.backward()

            label.fill_(fake_label)
            output = neta(noassd)
            lossA_real2 = F.binary_cross_entropy(output, label)
            lossA += lossA_real2.item()
            lossA_real2.backward()

            label.fill_(fake_label)
            output = neta(faked)
            lossA_fake = F.binary_cross_entropy(output, label)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()
        
        if i % n_d ==0: 
            # update G
            lossG = 0
            optimG.zero_grad()
            fake, A = netg(img)
            output = netd(fake)

            label.fill_(real_label)
            lossGD = F.binary_cross_entropy(output, label)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)  ##這裡需要retain graph 因為他之後有需要fake，因此需要retain

            faked = th.cat((img, fake), 1)
            output = neta(faked)
            label.fill_(real_label)
            lossGA = F.binary_cross_entropy(output, label)
            lossG += lossGA.item()
            lossGA.backward()

            ## new tripletloss
            _, P = netg(ass_img)
            __, N = netg(noass_img)
            lossf = TripletLoss(margin) #因為之前的A都沒被使用過所以不會遺失，如果是拿fake ，LossGA就需要再retain graph
            lossTriplet =lossf(A, P, N)
            lossG += lossTriplet.item()
            lossTriplet.backward()

    #         ## new onlinetripletloss
    #         __, com = netg(com_img)
    #         loss_fn = OnlineTripletLoss(margin, SemihardNegativeTripletSelector(margin))
    #         lossTriplet,len_triplet = loss_fn(com, com_label)
    #         lossG += lossTriplet.item()
    #         lossTriplet.backward()
    # #         print(lossTriplet.item(),len_triplet)

    #         ## triplet loss
    #         __, P = netg(ass_img)
    #         __, N = netg(noass_img)
    #         lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin) #因為之前的A都沒被使用過所以不會遺失，如果是拿fake ，LossGA就需要再retain graph
    #         lossG += lossTriplet.item()
    #         lossTriplet.backward()
    # #         if i%10==0:
    # #             print("tripletloss ",lossTriplet.item())

            ## tripletloss no negative
    #         N_plus = th.zeros((A.size()), requires_grad=False).to(device)
    #         lossTriplet_AP = F.triplet_margin_loss(A, P, N_plus, margin = margin)
    #         lossG += lossTriplet_AP.item()
    #         lossTriplet += lossTriplet_AP
    #         lossTriplet.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG  Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
    


# from PixelDT code刪

In [ ]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir)

iteration = 0
lr = 0.0002
real_label = 1
fake_label = 0
batchSize = 128

real_label = th.zeros((batchSize, 1), requires_grad=False).to(device)
fake_label = th.ones((batchSize, 1), requires_grad=False).to(device)
optimG = optim.Adam(netg.parameters(), lr=lr)
optimD = optim.Adam(netd.parameters(), lr=lr)
optimA = optim.Adam(neta.parameters(), lr=lr)

print('Training starts')
while iteration < 40000:
    ass_label, noass_label, img = dataset.getbatch(batchSize)
    ass_label = ass_label.to(device).to(th.float32)
    noass_label = noass_label.to(device).to(th.float32)
    img = img.to(device).to(th.float32)
    
    # update D
    lossD = 0
    optimD.zero_grad()
    output = netd(ass_label)
#     label.fill_(real_label)
    lossD_real1 = F.binary_cross_entropy(output, real_label)
#     lossD += lossD_real1.item()
#     lossD_real1.backward()

#     label.fill_(real_label)
    output1 = netd(noass_label)
    lossD_real2 = F.binary_cross_entropy(output1, real_label)
#     lossD += lossD_real2.item()
#     lossD_real2.backward()

    fake = netg(img).detach()
#     label.fill_(fake_label)
    output2 = netd(fake)
    lossD_fake = F.binary_cross_entropy(output2, fake_label)
#     lossD += lossD_fake.item()
#     lossD_fake.backward()
    lossD = (lossD_real1+ lossD_real2+ lossD_fake)/3
    lossD.backward()

    lossD_item = lossD.item()
    optimD.step()
    
    # update A
    lossA = 0
    optimA.zero_grad()
    assd = th.cat((img, ass_label), 1)
    noassd = th.cat((img, noass_label), 1)
    fake = netg(img).detach()
    faked = th.cat((img, fake), 1)

#     label.fill_(real_label)
    output = neta(assd)
    lossA_real1 = F.binary_cross_entropy(output, real_label)
#     lossA += lossA_real1.item()
#     lossA_real1.backward()

#     label.fill_(fake_label)
    output1 = neta(noassd)
    lossA_real2 = F.binary_cross_entropy(output1, fake_label)
#     lossA += lossA_real2.item()
#     lossA_real2.backward()

#     label.fill_(fake_label)
    output2 = neta(faked)
    lossA_fake = F.binary_cross_entropy(output2, fake_label)
#     lossA += lossA_fake.item()
#     lossA_fake.backward()
    lossA = (lossA_real1+ lossA_real2 +lossA_fake)/3
    lossA.backward()
    
    lossA_item = lossA.item()
    optimA.step()
    
    # update G
    lossG = 0
    optimG.zero_grad()
    fake = netg(img)
    output = netd(fake)
#     label.fill_(real_label)
    lossGD = F.binary_cross_entropy(output, real_label)
#     lossG += lossGD.item()
#     lossGD.backward(retain_graph=True)

    faked = th.cat((img, fake), 1)
    output = neta(faked)
#     label.fill_(real_label)
    lossGA = F.binary_cross_entropy(output, real_label)
#     lossG += lossGA.item()
#     lossGA.backward()
    lossG = (lossGD + lossGA)/2
    lossG.backward()
    
    lossG_item = lossG.item()
    optimG.step()

    iteration += 1
    
    if iteration % 20 == 0:
        with th.no_grad():
            netg.eval()  #切換
            fake = netg(img) 
            netg.train() #切換回去
        fake = (fake + 1) / 2 * 255
        real = (ass_label + 1) / 2 * 255
        ori = (img + 1) / 2 * 255
        al = th.cat((fake, real, ori), 2)
        display = make_grid(al, 20).cpu().numpy()
        if win1 is None:
            win1 = vis.image(display,
                             opts=dict(title="train", caption='train'))
        else:
            vis.image(display, win=win1)
    if iteration % 5000 == 0 or iteration==500:
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_Name+'/snapshot'+ Model_Name +'_%d.t7' % iteration)
        with open(Model_Name+"/snapshot_log.txt", "a") as myfile:
            myfile.write('iter = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            iteration, lossG_item, lossA_item, lossD_item
        ))
        print('iter = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            iteration, lossG_item, lossA_item, lossD_item
        ))
    if iteration % 5000==0 or iteration==10 or iteration==500:
        if win is None:
            win = vis.line(X=np.array([[iteration, iteration,
                                        iteration]]),
                           Y=np.array([[lossG_item, lossA_item, lossD_item]]),
                           opts=dict(
                               title='GaitGAN',
                               ylabel='loss',
                               xlabel='iterations',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[iteration, iteration,
                                  iteration]]),
                     Y=np.array([[lossG_item, lossA_item, lossD_item]]),
                     win=win,
                     update='append')


# WGAN 

In [2]:
#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0
margin = 0
n_g = 1
n_d = 2
clip = 0.1


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)



label = th.zeros((batchSize, 1), requires_grad=False).to(device)
optimG = optim.RMSprop(netg.parameters(), lr=glr/2)
optimD = optim.RMSprop(netd.parameters(), lr=dlr/3)
optimA = optim.RMSprop(neta.parameters(), lr=dlr/3)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={},clip={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp, clip))

low_loss = 10

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        

        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        # update D
        if i % n_g==0:
            lossD = 0
            optimD.zero_grad()
            output = netd(ass_label)
    #         label.fill_(real_label)
    #         lossD_real1 = F.binary_cross_entropy(output, label)
            lossD_real1 = -th.mean(output)
            lossD += lossD_real1.item()
            lossD_real1.backward()

    #         label.fill_(real_label)
            output1 = netd(noass_label)
    #         lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD_real2 = -th.mean(output1)
            lossD += lossD_real2.item()
            lossD_real2.backward()

            fake = netg(img).detach()
    #         label.fill_(fake_label)
            output2 = netd(fake)
    #         lossD_fake = F.binary_cross_entropy(output2, label)
            lossD_fake = th.mean(output2)
            lossD += lossD_fake.item()
            lossD_fake.backward()

            optimD.step()

            for p in netd.parameters():
                p.data.clamp_(-clip, clip)

            # update A
            lossA = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

    #         label.fill_(real_label)
            output1 = neta(assd)
    #         lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA_real1 = -th.mean(output1)
            lossA += lossA_real1.item()
            lossA_real1.backward()

    #         label.fill_(fake_label)
            output = neta(noassd)
    #         lossA_real2 = F.binary_cross_entropy(output, label)
            lossA_real2 = th.mean(output)
            lossA += lossA_real2.item()
            lossA_real2.backward()

    #         label.fill_(fake_label)
            output = neta(faked)
    #         lossA_fake = F.binary_cross_entropy(output, label)
            lossA_fake = th.mean(output)
            lossA += lossA_fake.item()
            lossA_fake.backward()

            optimA.step()

            for p in neta.parameters():
                p.data.clamp_(-clip, clip)

        # update G
        if i % n_d == 0:
            lossG = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

#             label.fill_(real_label)
#             lossGD = F.binary_cross_entropy(output, label)
            lossGD = -th.mean(output)
            lossG += lossGD.item()
            lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
#             label.fill_(real_label)
#             lossGA = F.binary_cross_entropy(output, label)
            lossGA = -th.mean(output)
            lossG += lossGA.item()
            lossGA.backward()

            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    
    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))        
        
        


n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts
Epoch = 2, ErrG = 3.752237558364868, ErrA = -1.8985527356465657, ErrD = -3.5465147892634072
Epoch = 4, ErrG = 5.705430269241333, ErrA = -2.807316621144613, ErrD = -5.611066023508708
Epoch = 6, ErrG = 4.896071195602417, ErrA = -3.07237180074056, ErrD = -6.601948658625285
Epoch = 8, ErrG = 8.87260103225708, ErrA = -4.403616587320964, ErrD = -9.87374464670817
Epoch = 10, ErrG = 7.947443723678589, ErrA = -4.1470723152160645, ErrD = -8.064921021461487
Epoch = 12, ErrG = 1.238088607788086, ErrA = -4.921435038248698, ErrD = -4.710442225138347
Epoch = 14, ErrG = 0.5194168090820312, ErrA = -5.6255998611450195, ErrD = -5.323735237121582
Epoch = 16, ErrG = 6.348399639129639, ErrA = -7.5277970631917315, ErrD = -7.016017913818359
Epoch = 18, ErrG = 0.8842306137084961, ErrA = -7.25491460164388, ErrD = -7.062686284383138
Epoch = 20, ErrG = 0.0035724639892578125, ErrA = -7.803771336873372, ErrD = -7.927731831868489
Epoch = 22, E

Epoch = 178, ErrG = 13.589683532714844, ErrA = -53.47724533081055, ErrD = -60.667805989583336
Epoch = 180, ErrG = 27.157360076904297, ErrA = -54.507459004720054, ErrD = -51.485514322916664
Epoch = 182, ErrG = 56.5279426574707, ErrA = -54.72101338704427, ErrD = -54.934326171875
Epoch = 184, ErrG = 26.18047523498535, ErrA = -55.34730529785156, ErrD = -59.38808059692383
Epoch = 186, ErrG = 13.245643615722656, ErrA = -56.8813362121582, ErrD = -60.9880002339681
Epoch = 188, ErrG = 14.880294799804688, ErrA = -53.91685994466146, ErrD = -41.10221862792969
Epoch = 190, ErrG = 33.4071044921875, ErrA = -55.473968505859375, ErrD = -54.388196309407554
Epoch = 192, ErrG = 54.02478504180908, ErrA = -50.48332722981771, ErrD = -56.40521113077799
Epoch = 194, ErrG = 35.50703430175781, ErrA = -58.89679463704427, ErrD = -55.31469599405924
Epoch = 196, ErrG = 47.07379150390625, ErrA = -56.774068196614586, ErrD = -64.39537811279297
Epoch = 198, ErrG = 8.856361389160156, ErrA = -59.77715301513672, ErrD = -64

Epoch = 356, ErrG = 33.6126594543457, ErrA = -54.70630900065104, ErrD = -60.700757344563804
Epoch = 358, ErrG = 38.190752029418945, ErrA = -54.00933074951172, ErrD = -58.47474924723307
Epoch = 360, ErrG = 24.69669532775879, ErrA = -51.98524602254232, ErrD = -69.35917663574219
Epoch = 362, ErrG = 20.46735382080078, ErrA = -58.84796142578125, ErrD = -65.61435190836589
Epoch = 364, ErrG = 32.753963470458984, ErrA = -54.45555623372396, ErrD = -72.78407796223958
Epoch = 366, ErrG = 50.228535652160645, ErrA = -63.61809285481771, ErrD = -68.19214502970378
Epoch = 368, ErrG = 35.94788932800293, ErrA = -55.87086486816406, ErrD = -63.45636494954427
Epoch = 370, ErrG = 13.586647033691406, ErrA = -55.783398946126304, ErrD = -54.916760762532554
Epoch = 372, ErrG = 17.255874633789062, ErrA = -48.30359903971354, ErrD = -46.36835225423177
Epoch = 374, ErrG = 32.9892520904541, ErrA = -55.58989715576172, ErrD = -59.28937530517578
Epoch = 376, ErrG = 22.925832748413086, ErrA = -53.68057378133138, ErrD = 

Epoch = 534, ErrG = 35.67050743103027, ErrA = -64.6280288696289, ErrD = -75.14555549621582
Epoch = 536, ErrG = 46.15334701538086, ErrA = -52.71881357828776, ErrD = -65.6470947265625
Epoch = 538, ErrG = 28.56289291381836, ErrA = -67.59828821818034, ErrD = -64.95235061645508
Epoch = 540, ErrG = 29.25141143798828, ErrA = -51.203712463378906, ErrD = -58.197113037109375
Epoch = 542, ErrG = 42.39829158782959, ErrA = -52.20666249593099, ErrD = -70.6221415201823
Epoch = 544, ErrG = 0.5973739624023438, ErrA = -60.515010833740234, ErrD = -52.27333068847656
Epoch = 546, ErrG = 56.850443840026855, ErrA = -64.28490193684895, ErrD = -68.94095865885417
Epoch = 548, ErrG = 48.342369079589844, ErrA = -63.897132873535156, ErrD = -73.07439422607422
Epoch = 550, ErrG = 20.176572799682617, ErrA = -54.681190490722656, ErrD = -57.4961903889974
Epoch = 552, ErrG = 27.500083923339844, ErrA = -62.00841776529948, ErrD = -61.99379221598307
Epoch = 554, ErrG = 41.838890075683594, ErrA = -57.539103190104164, ErrD =

 # WGAN-GP

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00002
dlr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 10
beta1 = 0
beta2 = 0.999
margin = 0
n_g = 0
n_d = 5

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        
        # update D
        if i % n_g==0:
            lossD = 0
            lossD_ = 0
            optimD.zero_grad()
            output = netd(ass_label)
    #         label.fill_(real_label)
    #         lossD_real1 = F.binary_cross_entropy(output, label)
            lossD_real1 = -th.mean(output)
            lossD_ += lossD_real1
            lossD += lossD_real1.item()
    #         lossD_real1.backward()

    #         label.fill_(real_label)
            output1 = netd(noass_label)
    #         lossD_real2 = F.binary_cross_entropy(output1, label)
            lossD_real2 = -th.mean(output1)
            lossD_ += lossD_real2
            lossD += lossD_real2.item()
    #         lossD_real2.backward()

            fake = netg(img).detach()
    #         label.fill_(fake_label)
            output2 = netd(fake)
    #         lossD_fake = F.binary_cross_entropy(output2, label)
            lossD_fake = th.mean(output2)
            lossD_ += lossD_fake
            lossD += lossD_fake.item()
            gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
    #         lossD_fake.backward()
            lossD_ = lossD_/3 + lambda_gp * gradient_penalty
            lossD_.backward()

            optimD.step()


            # update A
            lossA = 0
            lossA_ = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            fake = netg(img).detach()
            faked = th.cat((img, fake), 1)

    #         label.fill_(real_label)
            output1 = neta(assd)
    #         lossA_real1 = F.binary_cross_entropy(output1, label)
            lossA_real1 = -th.mean(output1)
            lossA += lossA_real1.item()
            lossA_ += lossA_real1
    #         lossA_real1.backward()

    #         label.fill_(fake_label)
            output = neta(noassd)
    #         lossA_real2 = F.binary_cross_entropy(output, label)
            lossA_real2 = th.mean(output)
            lossA += lossA_real2.item()
            lossA_ += lossA_real2
    #         lossA_real2.backward()

    #         label.fill_(fake_label)
            output = neta(faked)
    #         lossA_fake = F.binary_cross_entropy(output, label)
            lossA_fake = th.mean(output)
            lossA += lossA_fake.item()
            lossA_ += lossA_fake
    #         lossA_fake.backward()
            gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
            lossA_ = lossA_/3 + lambda_gp * gradient_penalty

            lossA_.backward()
            optimA.step()
    
            
        # update G
        if i % n_d == 0:
            lossG = 0
            lossG_ = 0
            optimG.zero_grad()
            fake = netg(img)
            output = netd(fake)

#             label.fill_(real_label)
#             lossGD = F.binary_cross_entropy(output, label)
            lossGD = -th.mean(output)
            lossG += lossGD.item()
            lossG_ += lossGD
#             lossGD.backward(retain_graph=True)

            faked = th.cat((img, fake), 1)
            output = neta(faked)
#             label.fill_(real_label)
#             lossGA = F.binary_cross_entropy(output, label)
            lossGA = -th.mean(output)
            lossG += lossGA.item()
            lossG_ += lossGA
#             lossGA.backward()
            lossG_ = lossG_/2
            lossG_.backward(retain_graph=True)
            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))           

        
        


# SA GaitGAN (hing)

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        

epoches = 700
glr = 0.00001
dlr = 0.00004
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0.9
margin = 10
n_g = 2
n_d = 1


netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)

        if i % n_g ==0:
            # update D
            lossD = 0
            lossD_ = 0
            optimD.zero_grad()
            d_out_assreal,dr1 = netd(ass_label)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()

            lossD_ += d_loss_assreal
            lossD += d_loss_assreal.item()

            d_out_noassreal,dr2 = netd(noass_label)
            d_loss_noassreal = nn.ReLU()(1.0 - d_out_noassreal).mean()

            lossD_ += d_loss_noassreal
            lossD += d_loss_noassreal.item()

            fake, gf1 = netg(img)
            d_out_fake, df1 = netd(fake.detach())  #需要 detach 因為不希望更新fake的參數
            d_loss_fake = nn.ReLU()(1.0 + d_out_fake).mean()

            lossD_ += d_loss_fake
            lossD += d_loss_fake.item()
    #         gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
            lossD_ = lossD_/3
            lossD_.backward()
            optimD.step()


            # update A
            lossA = 0
            lossA_ = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            faked, gf1 = netg(img)
            faked = th.cat((img, faked.detach()), 1)  #需要 detach 因為不希望更新fake的參數

            d_out_assreal,dr1 = neta(assd)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()
            lossA += d_loss_assreal.item()
            lossA_ += d_loss_assreal

            d_out_noassreal,dr2 = neta(noassd)
            d_loss_noassreal = nn.ReLU()(1.0 + d_out_noassreal).mean()

            lossA_ += d_loss_noassreal
            lossA += d_loss_noassreal.item()

            d_out_faked, df3 = neta(faked)
            d_loss_faked = nn.ReLU()(1.0 + d_out_faked).mean()

            lossA_ += d_loss_faked
            lossA += d_loss_faked.item()
    #         gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
            lossA_ = lossA_/3
            lossA_.backward()
            optimA.step()

            
        # update G
        if i % n_d == 0:
            lossG = 0
            lossG_ = 0
            optimG.zero_grad()
            fake,_= netg(img)
            g_out_fake,_ = netd(fake)
            g_loss_fake = - g_out_fake.mean()

            lossG += g_loss_fake.item()
            lossG_ += g_loss_fake

            faked = th.cat((img, fake), 1)
            g_out_faked,_ = neta(faked)
            g_loss_faked = - g_out_faked.mean()
            lossG += g_loss_faked.item()
            lossG_ += g_loss_faked

            lossG_ = lossG_/2
            lossG_.backward(retain_graph=True) ##其實這裡也不需要retain graph 因為他只 backward 一次；
            #如果爾後有要再backward，第一次就需要retain graph
            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/2, lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))           

        

# SA GaitGAN (WGAN-GP) 刪

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00001
dlr = 0.00004
# lr = 0.00002
real_label = 1
fake_label = 0
batchSize = 32
n_critic = 0
target = '090'
lambda_gp = 10
beta1 = 0
beta2 = 0.9

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_critic = {}, target={},lambda_gp={} \n'.format(
            epoches, dlr, glr, batchSize, beta1, beta2, n_critic, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
    for i, (ass_label, noass_label, img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        
        
        # update D
        lossD = 0
        lossD_ = 0
        optimD.zero_grad()
        d_out_assreal,dr1 = netd(ass_label)
        d_loss_assreal = -th.mean(d_out_assreal)

        lossD_ += d_loss_assreal
        lossD += d_loss_assreal.item()

        d_out_noassreal,dr2 = netd(noass_label)
        d_loss_noassreal = -th.mean(d_out_noassreal)

        lossD_ += d_loss_noassreal
        lossD += d_loss_noassreal.item()

        fake, gf1 = netg(img)
        d_out_fake, df1 = netd(fake.detach())  #需要 detach 因為不希望更新fake的參數
        d_loss_fake = th.mean(d_out_fake)

        lossD_ += d_loss_fake
        lossD += d_loss_fake.item()
        gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
        lossD_ = lossD_/3 + lambda_gp * gradient_penalty
#         lossD_ = lossD_/3
        lossD_.backward()
        optimD.step()
        
            
        # update A
        lossA = 0
        lossA_ = 0
        optimA.zero_grad()
        assd = th.cat((img, ass_label), 1)
        noassd = th.cat((img, noass_label), 1)
        faked, gf1 = netg(img)
        faked = th.cat((img, faked.detach()), 1)  #需要 detach 因為不希望更新fake的參數

        d_out_assreal,dr1 = neta(assd)
        d_loss_assreal = -th.mean(d_out_assreal)
        lossA += d_loss_assreal.item()
        lossA_ += d_loss_assreal

        d_out_noassreal,dr2 = neta(noassd)
        d_loss_noassreal = th.mean(d_out_noassreal)

        lossA_ += d_loss_noassreal
        lossA += d_loss_noassreal.item()
        
        d_out_faked, df3 = neta(faked)
        d_loss_faked = th.mean(d_out_faked)

        lossA_ += d_loss_faked
        lossA += d_loss_faked.item()
        gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
        lossA_ = lossA_/3 + lambda_gp * gradient_penalty
#         lossA_ = lossA_/3
        lossA_.backward()
        optimA.step()

            
        # update G
#         if i % n_critic == 0:
        lossG = 0
        lossG_ = 0
        optimG.zero_grad()
        fake,_= netg(img)
        g_out_fake,_ = netd(fake)
        g_loss_fake = - g_out_fake.mean()

        lossG += g_loss_fake.item()
        lossG_ += g_loss_fake

        faked = th.cat((img, fake), 1)
        g_out_faked,_ = neta(faked)
        g_loss_faked = - g_out_faked.mean()
        lossG += g_loss_faked.item()
        lossG_ += g_loss_faked
        
        lossG_ = lossG_/2
        lossG_.backward(retain_graph=True) ##其實這裡也不需要retain graph 因為他只 backward 一次；
        #如果爾後有要再backward，第一次就需要retain graph
        optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/2, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/2, lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/2)<low_loss:  
        low_loss = lossG/2
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/2, lossA/3, lossD/3
        ))           

        

# SA GaitGAN and triplet 

In [ ]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)

epoches = 700
glr = 0.00001
dlr = 0.00004
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0.9
margin = 5
n_g = 2
n_d = 1

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#         print("shape",ass_label.shape,noass_label.shape,img.shape)
        
    # while epoch < 30000:
    #     ass_label, noass_label, img = dataset.getbatch(batchSize)
        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)

        if i % n_g ==0:
            # update D
            lossD = 0
            lossD_ = 0
            optimD.zero_grad()
            d_out_assreal,dr1 = netd(ass_label)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()

            lossD_ += d_loss_assreal
            lossD += d_loss_assreal.item()

            d_out_noassreal,dr2 = netd(noass_label)
            d_loss_noassreal = nn.ReLU()(1.0 - d_out_noassreal).mean()

            lossD_ += d_loss_noassreal
            lossD += d_loss_noassreal.item()

            fake, code = netg(img)
            d_out_fake, df1 = netd(fake.detach())
            d_loss_fake = nn.ReLU()(1.0 + d_out_fake).mean()

            lossD_ += d_loss_fake
            lossD += d_loss_fake.item()
    #         gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
            lossD_ = lossD_/3
            lossD_.backward()
            optimD.step()


            # update A
            lossA = 0
            lossA_ = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            faked, code = netg(img)
            faked = th.cat((img, faked.detach()), 1)

            d_out_assreal,dr1 = neta(assd)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()
            lossA += d_loss_assreal.item()
            lossA_ += d_loss_assreal

            d_out_noassreal,dr2 = neta(noassd)
            d_loss_noassreal = nn.ReLU()(1.0 + d_out_noassreal).mean()

            lossA_ += d_loss_noassreal
            lossA += d_loss_noassreal.item()

            d_out_faked, df3 = neta(faked)
            d_loss_faked = nn.ReLU()(1.0 + d_out_faked).mean()

            lossA_ += d_loss_faked
            lossA += d_loss_faked.item()
    #         gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
            lossA_ = lossA_/3
            lossA_.backward()
            optimA.step()

            
        # update G
        if i % n_d == 0:
            lossG = 0
            lossG_ = 0
            optimG.zero_grad()
            fake, A= netg(img)
            g_out_fake,_ = netd(fake)
            g_loss_fake = - g_out_fake.mean()

            lossG += g_loss_fake.item()
            lossG_ += g_loss_fake

            faked = th.cat((img, fake), 1)
            g_out_faked,_ = neta(faked)
            g_loss_faked = - g_out_faked.mean()
            lossG += g_loss_faked.item()
            lossG_ += g_loss_faked

            # constrain on generator
            fake_ass, P = netg(ass_img)
            fake_noass, N = netg(noass_img)
            lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin)
            lossG_ += lossTriplet
            lossG += lossTriplet.item()
    #         lossTriplet.backward()

            lossG_ = lossG_/3
            lossG_.backward(retain_graph=True)
            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrTri = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/3,lossTriplet.item(), lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/3)<low_loss:  
        low_loss = lossG/3
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))           

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts


# SA GaitGAN and triplet (onlineloss)

In [2]:
%load_ext autoreload
%autoreload 2
from torch.autograd import grad, Variable

#python -m visdom.server
vis = visdom.Visdom(port=8097)
win = None
win1 = None
netg = NetG(nc=1)
netd = NetD(nc=1)
neta = NetA(nc=1)
device = th.device("cuda:1")

# weights init
all_mods = itertools.chain()
all_mods = itertools.chain(all_mods, [
    list(netg.children())[0].children(),
    list(netd.children())[0].children(),
    list(neta.children())[0].children()
])
for mod in all_mods:
    if isinstance(mod, nn.Conv2d) or isinstance(mod, nn.ConvTranspose2d):
#         init.xavier_normal_(tensor, gain=1.)
        init.normal_(mod.weight, 0.0, 0.02)
    elif isinstance(mod, nn.BatchNorm2d):
        init.normal_(mod.weight, 1.0, 0.02)
        init.constant_(mod.bias, 0.0)
        
epoches = 700
glr = 0.00001
dlr = 0.00004
real_label = 1
fake_label = 0
batchSize = 32
target = '090'
lambda_gp = 0
beta1 = 0
beta2 = 0.9
margin = 10
n_g = 2
n_d = 1

netg = netg.to(device)
netd = netd.to(device)
neta = neta.to(device)
netg.train()
netd.train()
neta.train()
dataset = CASIABDataset(data_dir=Data_Dir,target=target)
train_loader = th.utils.data.DataLoader(dataset, batch_size=batchSize, shuffle=True, num_workers=2, pin_memory=False)


optimG = optim.Adam(netg.parameters(), lr=glr, betas=(beta1, beta2))
optimD = optim.Adam(netd.parameters(), lr=dlr, betas=(beta1, beta2))
optimA = optim.Adam(neta.parameters(), lr=dlr, betas=(beta1, beta2))
# optimG = optim.RMSprop(netg.parameters(), lr=lr)
# optimD = optim.RMSprop(netd.parameters(), lr=lr)
# optimA = optim.RMSprop(neta.parameters(), lr=lr)

print("write parameter log...")
with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, margin = {}, dlr = {}, glr={}, batchsize = {}, beta1={}, beta2={}, n_d = {}, n_g={} target={},lambda_gp={} \n'.format(
            epoches, margin, dlr, glr, batchSize, beta1, beta2, n_d, n_g, target, lambda_gp))

low_loss = 10

def compute_gradient_penalty(D, real_samples, fake_samples):
    """Calculates the gradient penalty loss for WGAN GP"""
    # Random weight term for interpolation between real and fake samples
#     alpha = Tensor(np.random.random((real_samples.size(0), 1, 1, 1)))
    alpha = th.rand((batchSize, 1, 1, 1)).to(device).to(th.float32)
    # Get random interpolation between real and fake samples
    interpolates = (alpha * real_samples + ((1 - alpha) * fake_samples)).requires_grad_(True)

    d_interpolates = D(interpolates)
#     fake = Variable(Tensor(real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
    gradients = grad(outputs=d_interpolates, 
                     inputs=interpolates, 
                     grad_outputs=th.ones([real_samples.shape[0],1]).to(device).requires_grad_(False),
#                      grad_outputs = fake,
                     create_graph=True, 
                     retain_graph=True, 
                     only_inputs=True)[0]
#     fake = Variable((real_samples.shape[0], 1).fill_(1.0), requires_grad=False)
#     # Get gradient w.r.t. interpolates
#     gradients = autograd.grad(
#         outputs=d_interpolates,
#         inputs=interpolates,
#         grad_outputs=fake,
#         create_graph=True,
#         retain_graph=True,
#         only_inputs=True,
#     )[0]
    gradients = gradients.view(gradients.size(0), -1)
    gradient_penalty = ((gradients.norm(2, dim=1) - 1) ** 2).mean()
    return gradient_penalty

print('Training starts')
for epoch in range(1,epoches+1):
#     for i, (ass_label, noass_label, img) in enumerate(train_loader):
    for i, (ass_label, noass_label, noass_img, img, ass_img) in enumerate(train_loader):
#     for i, (ass_label, noass_label, noass_img, img, ass_img, label_neg, label_anc, label_pos) in enumerate(train_loader):
        
#         com_img = th.cat((noass_img, img, ass_img), 0)
#         com_label = th.cat(( label_neg, label_anc, label_pos), 0)
#         com_img = com_img.to(device).to(th.float32)
#         com_label = com_label.to(device).to(th.float32)
# #         print("shape",ass_label.shape,noass_label.shape,img.shape)
        

        ass_label = ass_label.to(device).to(th.float32)
        noass_label = noass_label.to(device).to(th.float32)
        noass_img = noass_img.to(device).to(th.float32)
        img = img.to(device).to(th.float32)
        ass_img = ass_img.to(device).to(th.float32)

        if i % n_g ==0:
            # update D
            lossD = 0
            lossD_ = 0
            optimD.zero_grad()
            d_out_assreal,dr1 = netd(ass_label)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()

            lossD_ += d_loss_assreal
            lossD += d_loss_assreal.item()

            d_out_noassreal,dr2 = netd(noass_label)
            d_loss_noassreal = nn.ReLU()(1.0 - d_out_noassreal).mean()

            lossD_ += d_loss_noassreal
            lossD += d_loss_noassreal.item()

            fake, code = netg(img)
            d_out_fake, df1 = netd(fake.detach())
            d_loss_fake = nn.ReLU()(1.0 + d_out_fake).mean()

            lossD_ += d_loss_fake
            lossD += d_loss_fake.item()
    #         gradient_penalty = compute_gradient_penalty(netd, ass_label.data, fake.data)
            lossD_ = lossD_/3
            lossD_.backward()
            optimD.step()


            # update A
            lossA = 0
            lossA_ = 0
            optimA.zero_grad()
            assd = th.cat((img, ass_label), 1)
            noassd = th.cat((img, noass_label), 1)
            faked, code = netg(img)
            faked = th.cat((img, faked.detach()), 1)

            d_out_assreal,dr1 = neta(assd)
            d_loss_assreal = nn.ReLU()(1.0 - d_out_assreal).mean()
            lossA += d_loss_assreal.item()
            lossA_ += d_loss_assreal

            d_out_noassreal,dr2 = neta(noassd)
            d_loss_noassreal = nn.ReLU()(1.0 + d_out_noassreal).mean()

            lossA_ += d_loss_noassreal
            lossA += d_loss_noassreal.item()

            d_out_faked, df3 = neta(faked)
            d_loss_faked = nn.ReLU()(1.0 + d_out_faked).mean()

            lossA_ += d_loss_faked
            lossA += d_loss_faked.item()
    #         gradient_penalty = compute_gradient_penalty(neta, assd.data, faked.data)
            lossA_ = lossA_/3
            lossA_.backward()
            optimA.step()


            
        # update G
        if i % n_d == 0:
            lossG = 0
            lossG_ = 0
            optimG.zero_grad()
            fake, A= netg(img)
            g_out_fake,_ = netd(fake)
            g_loss_fake = - g_out_fake.mean()

            lossG += g_loss_fake.item()
            lossG_ += g_loss_fake

            faked = th.cat((img, fake), 1)
            g_out_faked,_ = neta(faked)
            g_loss_faked = - g_out_faked.mean()
            lossG += g_loss_faked.item()
            lossG_ += g_loss_faked

    #         # constrain on encoder
    #         fake_ass, P = netg(ass_img)
    #         fake_noass, N = netg(noass_img)
    #         lossTriplet = F.triplet_margin_loss(A, P, N, margin = margin)
    #         lossG_ += lossTriplet
    #         lossG += lossTriplet.item()
    # #         lossTriplet.backward()

            ## new tripletloss
            fake_ass, P = netg(ass_img)
            fake_noass, N = netg(noass_img)
            loss_fn = TripletLoss(margin)
            lossTriplet = loss_fn(A, P, N)
            lossG += lossTriplet.item()
            lossG_ += lossTriplet
            
            ## new onlinetripletloss(semi-hard/ hard)
#             __, com = netg(com_img)
#             loss_fn = OnlineTripletLoss(margin, SemihardNegativeTripletSelector(margin))
#             lossTriplet,len_triplet = loss_fn(com, com_label)
#             lossG += lossTriplet.item()
#             lossG_ += lossTriplet


            lossG_ = lossG_/3
            lossG_.backward(retain_graph=True)
            optimG.step()

        if i % 20 == 0:
            with th.no_grad():
                netg.eval()  #切換
                fake,_ = netg(img) 
                netg.train() #切換回去
            fake = (fake + 1) / 2 * 255
            real = (ass_label + 1) / 2 * 255
            ori = (img + 1) / 2 * 255
            al = th.cat((fake, real, ori), 2)
            display = make_grid(al, 20).cpu().numpy()
            if win1 is None:
                win1 = vis.image(display,
                                 opts=dict(title="train", caption='train'))
            else:
                vis.image(display, win=win1)

    if epoch % 2==0:   #2   
        if win is None:
            win = vis.line(X=np.array([[epoch, epoch,
                                        epoch]]),
                           Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                           opts=dict(
                               title=Model_Name,
                               ylabel='loss',
                               xlabel='epochs',
                               legend=['lossG', 'lossA', 'lossD']
                           ))
        else:
            vis.line(X=np.array([[epoch, epoch,
                                  epoch]]),
                     Y=np.array([[lossG/3, lossA/3, lossD/3]]),
                     win=win,
                     update='append')

        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))
        print('Epoch = {}, ErrG = {}, ErrTri = {}, ErrA = {}, ErrD = {}, Gattn={}, Dattn={}, Aattn={}'.format(
            epoch, lossG/3,lossTriplet.item(), lossA/3, lossD/3, netg.attn.gamma.item(), netd.attn.gamma.item(), neta.attn.gamma.item()
        ))
            
    if (epoch>= 300) and epoch%20==0:      
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/snapshot'+ Model_Name +'_%d.t7' % epoch)
        
    if (epoch>= 550) and (lossG/3)<low_loss:  
        low_loss = lossG/3
        state = {
            'netA': neta.state_dict(),
            'netG': netg.state_dict(),
            'netD': netd.state_dict()
        }
        th.save(state, Model_dir+'/lowest_snapshot'+ Model_Name +'_%d.t7' % epoch)
        with open(Model_dir+"/snapshot_log.txt", "a") as myfile:
            myfile.write('lower_lossG Epoch = {}, ErrG = {}, ErrA = {}, ErrD = {} \n'.format(
            epoch, lossG/3, lossA/3, lossD/3
        ))           

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
n_con= 10 ,n_ang= 11
target =  090
write parameter log...
Training starts
Epoch = 2, ErrG = 2.2715400755405426, ErrTri = 5.083058834075928, ErrA = 0.6132369836171468, ErrD = 0.42663437128067017, Gattn=-0.004093634895980358, Dattn=0.001592523418366909, Aattn=-0.00026734417770057917
Epoch = 4, ErrG = 1.7833725412686665, ErrTri = 3.355182647705078, ErrA = 0.6522229413191477, ErrD = 0.2876933229466279, Gattn=-0.007606056518852711, Dattn=0.002421888057142496, Aattn=-0.0007175184437073767
Epoch = 6, ErrG = 1.9610540866851807, ErrTri = 3.2841928005218506, ErrA = 0.45963972186048824, ErrD = 0.16932998845974603, Gattn=-0.00994974933564663, Dattn=0.005251366179436445, Aattn=-0.0009563870844431221
Epoch = 8, ErrG = 1.4270671606063843, ErrTri = 4.645321846008301, ErrA = 0.5926398995021979, ErrD = 0.7014000614484152, Gattn=-0.01099950261414051, Dattn=0.007787586655467749, Aattn=-0.0012496927520260215
Epoch = 10,

Epoch = 80, ErrG = 0.16324423253536224, ErrTri = 0.1723148375749588, ErrA = 0.6422685086727142, ErrD = 0.7343462308247884, Gattn=-0.015465529635548592, Dattn=0.056657616049051285, Aattn=-0.056079816073179245
Epoch = 82, ErrG = 0.14485337336858115, ErrTri = 0.376237154006958, ErrA = 0.6128157911201318, ErrD = 0.7411962455759445, Gattn=-0.015439232811331749, Dattn=0.057298894971609116, Aattn=-0.056913718581199646
Epoch = 84, ErrG = 0.6435860842466354, ErrTri = 1.2766661643981934, ErrA = 0.5652037287751833, ErrD = 0.5232811619838079, Gattn=-0.015394429676234722, Dattn=0.058205462992191315, Aattn=-0.05740742012858391
Epoch = 86, ErrG = 0.22223674257596335, ErrTri = 0.17306575179100037, ErrA = 0.642295648654302, ErrD = 0.44923944274584454, Gattn=-0.015399002470076084, Dattn=0.058931659907102585, Aattn=-0.05772387981414795
Epoch = 88, ErrG = 0.09850436449050903, ErrTri = 0.8105394244194031, ErrA = 0.5753927466770014, ErrD = 0.47322752078374225, Gattn=-0.01538153737783432, Dattn=0.05951492860

Epoch = 160, ErrG = 0.2165762037038803, ErrTri = 0.1756746619939804, ErrA = 0.5081679473320643, ErrD = 0.47634710371494293, Gattn=-0.0152118019759655, Dattn=0.08104931563138962, Aattn=-0.07209422439336777
Epoch = 162, ErrG = 0.2731326222419739, ErrTri = 0.4268786907196045, ErrA = 0.51544588804245, ErrD = 0.6217548524339994, Gattn=-0.015214385464787483, Dattn=0.0815313458442688, Aattn=-0.07256826013326645
Epoch = 164, ErrG = 0.45817799121141434, ErrTri = 0.37459129095077515, ErrA = 0.6018440673748652, ErrD = 0.6496713428447644, Gattn=-0.015200850553810596, Dattn=0.08200043439865112, Aattn=-0.0729425698518753
Epoch = 166, ErrG = 0.04504639903704325, ErrTri = 0.30640488862991333, ErrA = 0.5552511215209961, ErrD = 0.5583118001619974, Gattn=-0.015203678049147129, Dattn=0.08249927312135696, Aattn=-0.07338415831327438
Epoch = 168, ErrG = 0.6910771330197653, ErrTri = 0.2950499653816223, ErrA = 0.578738946467638, ErrD = 0.31170251220464706, Gattn=-0.015208196826279163, Dattn=0.08314571529626846

Epoch = 240, ErrG = 0.4526013682285945, ErrTri = 0.19783402979373932, ErrA = 0.48766593138376874, ErrD = 0.3891867697238922, Gattn=-0.015054885298013687, Dattn=0.09935171902179718, Aattn=-0.08996959030628204
Epoch = 242, ErrG = 0.3864154815673828, ErrTri = 0.17907527089118958, ErrA = 0.41329756875832874, ErrD = 0.4193260173002879, Gattn=-0.01504947617650032, Dattn=0.09979476034641266, Aattn=-0.09050522744655609
Epoch = 244, ErrG = 0.6113575994968414, ErrTri = 0.31043198704719543, ErrA = 0.4071834360559781, ErrD = 0.38404718848566216, Gattn=-0.015036088414490223, Dattn=0.10009735822677612, Aattn=-0.0911499634385109
Epoch = 246, ErrG = 0.26896525422732037, ErrTri = 0.2737775146961212, ErrA = 0.48321080952882767, ErrD = 0.5130998392899832, Gattn=-0.015044884756207466, Dattn=0.1004670038819313, Aattn=-0.09167250245809555
Epoch = 248, ErrG = 0.6412517428398132, ErrTri = 0.25145840644836426, ErrA = 0.4366607088595629, ErrD = 0.25966260582208633, Gattn=-0.015025623142719269, Dattn=0.100872032

Epoch = 320, ErrG = 0.7262199819087982, ErrTri = 0.29892340302467346, ErrA = 0.4301841954390208, ErrD = 0.15722828917205334, Gattn=-0.014904887415468693, Dattn=0.11397851258516312, Aattn=-0.11090515553951263
Epoch = 322, ErrG = 0.4557248502969742, ErrTri = 0.04662661254405975, ErrA = 0.33638131618499756, ErrD = 0.21228575458129248, Gattn=-0.01490264106541872, Dattn=0.11437119543552399, Aattn=-0.11152851581573486
Epoch = 324, ErrG = 0.511737714211146, ErrTri = 0.27237382531166077, ErrA = 0.428065021832784, ErrD = 0.1517817129691442, Gattn=-0.014896749518811703, Dattn=0.11459377408027649, Aattn=-0.11210143566131592
Epoch = 326, ErrG = 0.6331769227981567, ErrTri = 0.08537405729293823, ErrA = 0.4065566708644231, ErrD = 0.38505323727925617, Gattn=-0.01490033883601427, Dattn=0.11491291970014572, Aattn=-0.11247614026069641
Epoch = 328, ErrG = 0.3078423887491226, ErrTri = 0.2863078713417053, ErrA = 0.4195273605485757, ErrD = 0.30084332823753357, Gattn=-0.014885512180626392, Dattn=0.11526453495

Epoch = 400, ErrG = 0.6671606798966726, ErrTri = 0.32419314980506897, ErrA = 0.3439147596557935, ErrD = 0.21877022087574005, Gattn=-0.014835789799690247, Dattn=0.12651307880878448, Aattn=-0.13783110678195953
Epoch = 402, ErrG = 0.6959218035141627, ErrTri = 0.18467874825000763, ErrA = 0.3573266689976056, ErrD = 0.11654679415126641, Gattn=-0.014822975732386112, Dattn=0.1268615424633026, Aattn=-0.13814117014408112
Epoch = 404, ErrG = 0.6553886582454046, ErrTri = 0.16939111053943634, ErrA = 0.44069134443998337, ErrD = 0.27583343784014386, Gattn=-0.014820008538663387, Dattn=0.12716928124427795, Aattn=-0.13852089643478394
Epoch = 406, ErrG = 0.1907974829276403, ErrTri = 0.07312677800655365, ErrA = 0.35742249588171643, ErrD = 0.2782003978888194, Gattn=-0.014832843095064163, Dattn=0.12737083435058594, Aattn=-0.13938060402870178
Epoch = 408, ErrG = 0.4375511705875397, ErrTri = 0.1487484872341156, ErrA = 0.3895813276370366, ErrD = 0.25210554525256157, Gattn=-0.014794943854212761, Dattn=0.1277206

Epoch = 480, ErrG = 0.726226852585872, ErrTri = 0.0472123809158802, ErrA = 0.47799859444300336, ErrD = 0.12064676235119502, Gattn=-0.014843268319964409, Dattn=0.1376672387123108, Aattn=-0.16839942336082458
Epoch = 482, ErrG = 0.4924769476056099, ErrTri = 0.08704926818609238, ErrA = 0.3435625309745471, ErrD = 0.3399594575166702, Gattn=-0.014844181947410107, Dattn=0.1378173977136612, Aattn=-0.16893552243709564
Epoch = 484, ErrG = 0.6846865378320217, ErrTri = 0.059720564633607864, ErrA = 0.40098026146491367, ErrD = 0.1631354739268621, Gattn=-0.014818742871284485, Dattn=0.1379549652338028, Aattn=-0.16942249238491058
Epoch = 486, ErrG = 0.6026637877027193, ErrTri = 0.040529765188694, ErrA = 0.42076122015714645, ErrD = 0.25067611535390216, Gattn=-0.014828508719801903, Dattn=0.1383891999721527, Aattn=-0.1703626662492752
Epoch = 488, ErrG = 0.7364134192466736, ErrTri = 0.06254643201828003, ErrA = 0.4094451268513997, ErrD = 0.16428804397583008, Gattn=-0.014821958728134632, Dattn=0.1385522186756

Epoch = 560, ErrG = 0.9181796908378601, ErrTri = 0.3365482687950134, ErrA = 0.28590088337659836, ErrD = 0.15627321725090346, Gattn=-0.014788860455155373, Dattn=0.14669159054756165, Aattn=-0.19761358201503754
Epoch = 562, ErrG = 0.7749171828230222, ErrTri = 0.08283042162656784, ErrA = 0.43818465868632, ErrD = 0.19554249321420988, Gattn=-0.014783713966608047, Dattn=0.14681266248226166, Aattn=-0.19808261096477509
Epoch = 564, ErrG = 0.7691735823949178, ErrTri = 0.3236265182495117, ErrA = 0.25681896321475506, ErrD = 0.20386691888173422, Gattn=-0.014772050082683563, Dattn=0.14706161618232727, Aattn=-0.1991758942604065
Epoch = 566, ErrG = 0.8823995192845663, ErrTri = 0.32675981521606445, ErrA = 0.27666106820106506, ErrD = 0.09451251104474068, Gattn=-0.014788221567869186, Dattn=0.14744187891483307, Aattn=-0.19968156516551971
Epoch = 568, ErrG = 0.7342492217818896, ErrTri = 0.1038106307387352, ErrA = 0.33737077315648395, ErrD = 0.04258691271146139, Gattn=-0.014806964434683323, Dattn=0.14758655

Epoch = 640, ErrG = 0.5829573273658752, ErrTri = 0.12776216864585876, ErrA = 0.34545545776685077, ErrD = 0.06914480465153854, Gattn=-0.014727042056620121, Dattn=0.1544439047574997, Aattn=-0.2237456887960434
Epoch = 642, ErrG = 0.609822690486908, ErrTri = 0.29725295305252075, ErrA = 0.2481230596701304, ErrD = 0.08848354034125805, Gattn=-0.014725122600793839, Dattn=0.15474635362625122, Aattn=-0.22378361225128174
Epoch = 644, ErrG = 0.7339262713988622, ErrTri = 0.1621600240468979, ErrA = 0.34866116878887016, ErrD = 0.068463580061992, Gattn=-0.014705088920891285, Dattn=0.15490077435970306, Aattn=-0.22417311370372772
Epoch = 646, ErrG = 0.660824162264665, ErrTri = 0.049027152359485626, ErrA = 0.37681002418200177, ErrD = 0.09153017650047938, Gattn=-0.014694875106215477, Dattn=0.15498563647270203, Aattn=-0.22491084039211273
Epoch = 648, ErrG = 0.7591326534748077, ErrTri = 0.2632124722003937, ErrA = 0.24088914692401886, ErrD = 0.1717893679936727, Gattn=-0.014687495306134224, Dattn=0.1553063541